<a href="https://colab.research.google.com/github/ksoliman/DataCamp/blob/master/Copy_of_Project_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Demo

This demo is designed to provide some general tips and tricks for the ITDS Fall 2019 project. For full details on the project, please refer to [the project requirements](https://grantmlong.com/teaching/fall2019/project/Project-ITDS-Fall-2019.pdf).

***
This demo uses `scikit-learn`, but by no means are you required to use this or any other particular package.



In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [0]:
train_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_train.csv', index_col=0)
test_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test1.csv', index_col=0)
submit1_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test2.csv', index_col=0)
submit2_df = pd.read_csv('https://grantmlong.com/data/SE_rents2018_test3.csv', index_col=0)

In [0]:
train_df.sample(5).transpose()
#bbl and bin helps w city data

rental_id,7134850,7203613,7219147,7159297,7150303
addr_unit,#3,#PH8FL,#3,#9B,#615
building_id,1282774,22102,1237375,60493,457
bedrooms,2,3,2,2,0
bathrooms,1,2,1,2,1
size_sqft,800,4000,750,869,645
created_at,2018-05-01 15:34:58,2018-05-23 18:02:21,2018-05-30 12:11:27,2018-05-08 18:07:33,2018-05-05 14:55:15
addr_street,22-18 47 STREET,54 BLEECKER STREET,10-23 47 ROAD,393 WEST END AVENUE,55 WALL STREET
addr_city,Astoria,New York,Long Island City,New York,New York
addr_zip,11105,10012,11101,10024,10005
addr_lat,40.7695,40.7258,40.7448,40.7842,40.7058


#### Build training data

In [0]:
# extract usable features
feature_cols = [
    'bedrooms', 'year_built', 'bathrooms', 'min_to_subway', 
    'size_sqft', 'no_fee', 'has_doorman'
]
train_features = train_df[feature_cols] 

# impute missing values with medians
train_features = train_features.fillna(train_features.median(), axis=0)

# construct target vector
train_target = train_df['rent']

#### Fit model

In [0]:
lreg = LinearRegression()
lreg.fit(train_features, train_target)


#### Predict and Measure Using Test 1

In [0]:
test_features = test_df[feature_cols] 

# impute missing values with medians
test_features = test_features.fillna(train_features.median(), axis=0)

# construct predictions 
test_df['predicted'] = lreg.predict(test_features)

mean_squared_error(test_df['rent'], test_df['predicted'])

#### Combine Data, Predict Values for Test 2

In [0]:
master_df = train_df.append(test_df, sort=False)

master_features = master_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
master_target = master_df['rent']

lreg.fit(master_features, master_target)


#### Create Submission File for `test2`

In [0]:
submit1_features = submit1_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit1_df['predictions'] = lreg.predict(submit1_features)
submit1_df['predictions'].to_csv('sample_submission1.csv', header=True)

submit1_df['fake_rent'] = np.ones(submit1_df['predictions'].shape) * master_target.median()
mean_squared_error(submit1_df['predictions'], submit1_df['fake_rent'])

#### Create Submission File for `test3`

In [0]:
submit2_features = submit2_df[feature_cols].fillna(master_df[feature_cols].median(), axis=0)
submit2_df['predictions'] = lreg.predict(submit2_features)
submit2_df['predictions'].to_csv('sample_submission2.csv', header=True)

submit2_df['fake_rent'] = np.ones(submit2_df['predictions'].shape) * master_target.median()
mean_squared_error(submit2_df['predictions'], submit2_df['fake_rent'])